In [1]:
%matplotlib inline
import json
import pandas
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [2]:
with open('./results/overlap_data.json') as f:
    data = json.load(f)
overlap_df = pandas.DataFrame(data)
overlap_df['max_test_all_match_prop'] = overlap_df.max_test_long_match/overlap_df.max_test_len
overlap_df['max_code_all_match_prop'] = overlap_df.max_code_long_match/overlap_df.max_code_len
overlap_df['project'] = overlap_df.bug_name.map(lambda x: x.split('_')[0])
overlap_df.head()

,bug_name,max_test_proportion,max_test_long_match,max_test_len,sel_test_raw_len,max_code_proportion,max_code_long_match,max_code_len,sel_code_raw_len,max_test_all_match_prop,max_code_all_match_prop,project
0,Math_89,0.973134,300,335,456,0.000000,0,76,112,0.895522,0.000000,Math
1,Math_42,0.983845,253,619,698,0.883013,571,1248,1548,0.408724,0.457532,Math
2,Math_102,0.745157,200,1652,2762,0.903172,250,599,780,0.121065,0.417362,Math
3,Math_105,0.778146,203,302,372,0.000000,0,77,94,0.672185,0.000000,Math
4,Math_45,0.606557,50,122,143,0.919431,100,211,252,0.409836,0.473934,Math


In [3]:
THR = 0.9
print('[Test] Overall overlap mean:', (overlap_df.max_test_proportion > THR).sum())
print('[Test] Complete overlap mean:', (overlap_df.max_test_all_match_prop > THR).sum())
print('[Code] Overall overlap mean:', (overlap_df.max_code_proportion > THR).sum())
print('[Code] Complete overlap mean:', (overlap_df.max_code_all_match_prop > THR).sum())
print('Bug compromised:', ((overlap_df.max_test_proportion > 0.9) | (overlap_df.max_code_proportion > 0.9)).sum())

[Test] Overall overlap mean: 123
[Test] Complete overlap mean: 78
[Code] Overall overlap mean: 136
[Code] Complete overlap mean: 50
Bug compromised: 205


In [4]:
compromised_count = \
    Counter(overlap_df[(overlap_df.max_test_proportion > 0.9) | (overlap_df.max_code_proportion > 0.9)].project)
overall_count = \
    Counter(overlap_df.project)
data = []
for key in compromised_count:
    data.append({
        'project': key,
        'total': overall_count[key],
        'compromised_prop': compromised_count[key]/overall_count[key]
    })
pandas.DataFrame(data)

,project,total,compromised_prop
0,Math,104,0.673077
1,Time,25,0.920000
2,Closure,130,0.353846
3,Lang,61,0.786885
4,Chart,25,0.720000
